# Tutorial on SAT Solving (DPLL)

This is a tutorial on using the Davis-Putnam-Logemann-Loveland (DPLL) Algorithm to solve Boolean Satisfiability problems. Many codes in this tutorial can be found (together with references and acknowledgements) in the .py software version in the github: https://github.com/zhaoy37/SAT_Solver. The user can also follow the readme tutorial on the github. We leave any evaluations and solver comparisons to the python version part of the project in case the user is interested.

## Background on SAT solvers.

Imagine that you are provided with a boolean formula P and asked to determine if P is satisfiable. P is satisfiabile if there exists values for each variable such that P evaluates to true. For example, the formula $P = (x1 \wedge x2) \vee x3$ is satisfiable because there exists a model {x1 = 1, x2 = 1, x3 = 0} such that the the boolean formula evaluates to true. On the contrary, $P = (x1 \wedge \neg x1)$ is not satisfiable. Validity means that the formula evaluates to true for all models.

The implication of SAT solving is theoretically supported by the Cook-Levin Theorem, which states that Boolean SAT is NP-complete. Equivalently, any problem in NP can be reduced in polynomial time by a deterministic Turing Machine to a SAT problem. Therefore, many classical NP-complete problems in algorithms, such as graph coloring and sudoku, can be encoded into a SAT Problem representation and solved via a SAT solver.

## An Auxiliary Structure: A Logic Tree

The Davis-Putnam-Logemann-Loveland algorithm (DPLL) is a widely used algorithm in SAT solving. Its basic idea is recursive backtracking search of atom values that meet the satisfiability with some special heuristics. For the rest of the tutorial, we build a DPLL Solver from scratch.

We need to be able to represent SAT formulas in a data strucutre. To make things simple, we represent the formulas as lists. The strucuture is recursively determined. For instance, $P = (x1 \wedge x2) \vee x3$ can be represented as ["or", ["and", "x1", "x2"], "x3"]. The possible connectives include "or", "and", and "not".

To start with, to be able to quickly capture the properties of a SAT formula defined in list, we develop an auxiliary structure, which upon accepting the the list representation of the SAT formula, converts it to a tree representation in negative normal form (NNF), where the negations are directly in front of SAT atoms. We use the NNF logic tree to facilitate later procedure of the DPLL program.

In [1]:
class Logic:

    """
    Constructor

    The class takes in a list representation of the formula, and convert it to a
    logic tree representation. The converted data structure is a binary tree with
    connectives on the top. The left and right leaves are recursively defined binary
    trees or root atoms.
    """
    def __init__(self, formula):
        self.formula = formula
        self.left = None
        self.right = None
        self.value = self.__convert_formula_to_tree()
        self.leaves = self.load_leaves()
        self.pure_positives, self.pure_negatives = self.__find_pure_literals()


    """
    Perform an in-oder traversal of the tree for printing.
    This member function is majorly constructed for testing purposes.
    """
    def print_tree(self):
        if self.left is not None:
            self.left.print_tree()
        print(self.value)
        if self.right is not None:
            self.right.print_tree()

    """
    This member function finds the leaves of self. In this case, it finds the set
    of all SAT atoms.
    """
    def load_leaves(self):
        # Perform an in-order traversal.
        leaves = []
        if self.left is not None:
            leaves.extend(self.left.load_leaves())
        if (self.left is None) and (self.right is None):
            leaves.append(self.value)
        if self.right is not None:
            leaves.extend(self.right.load_leaves())
        leaves = [leaf for leaf in leaves if (leaf != "True" and leaf != "False")]
        return set(leaves)


    """
    This function finds the number of nodes of the tree.
    """
    def num_of_nodes(self):
        if self.left is None and self.right is None:
            return 1
        else:
            if self.left is None:
                return 1 + self.right.num_of_nodes()
            elif self.right is None:
                return 1 + self.left.num_of_nodes()
            else:
                return 1 + self.left.num_of_nodes() + self.right.num_of_nodes()


    """
    This method evaluates an assignment.
    The assignment is in the form of a dictionary mapping variables to boolean values.

    For instance, given that the formula P is ["and", "x1", "x2"] and an assignment
    {"x1" : 1, "x2" : 1}, the method should return true.
    """
    def evaluate(self, assignment):
        # Check if the assignment dictionary contains and only contains the leaves (excluding true and false) as the keys.
        if self.leaves != set(assignment.keys()):
            raise Exception("The given assignment is not valid")
        return self.__evaluate_assignment_kernel(assignment, self)


    """
    Private methods start here:
    """

    """
    This member function finds the literals of self.
    We use this function to facilitate the later DPLL algorithm.
    In the DPLL algorithm, we need to know the pure literals of a formula.
    We define the pure literals. The user may refer to the later portion of the material to
    see the definition of pure literals.
    """
    def __find_pure_literals(self):
        parents = dict()
        for leaf in self.leaves:
            parents[leaf] = []
        # Find the parents of the leaves.
        self.__find_parents_kernel(parents)
        # Find the pure positive and negative literals.
        pure_positives = [leaf for leaf in self.leaves if "not" not in parents[leaf]]
        pure_negatives = [leaf for leaf in self.leaves if (("not" in parents[leaf]) and len(parents[leaf]) == 1)]
        return pure_positives, pure_negatives


    """
    This member function is used in find_pure_literals.
    It finds the parents of the leaves.
    """
    def __find_parents_kernel(self, parents):
        if self.left is not None:
            if self.left.value in self.leaves:
                parents[self.left.value].append(self.value)
            self.left.__find_parents_kernel(parents)
        if self.right is not None:
            if self.right.value in self.leaves:
                parents[self.right.value].append(self.value)
            self.right.__find_parents_kernel(parents)


    """
    This method is the kernel to evaluate a potential assignment.
    The assignment is in the form of a dictionary mapping variables to boolean values.
    """
    def __evaluate_assignment_kernel(self, assignment, tree):
        if (tree.left is None) and (tree.right is None):
            if tree.value == "True":
                return True
            elif tree.value == "False":
                return False
            return assignment[tree.value]
        else:
            if tree.value == "not":
                return (not self.__evaluate_assignment_kernel(assignment, tree.right))
            elif tree.value == "and":
                return (self.__evaluate_assignment_kernel(assignment, tree.left) and
                        self.__evaluate_assignment_kernel(assignment, tree.right))
            else:
                return (self.__evaluate_assignment_kernel(assignment, tree.left) or
                        self.__evaluate_assignment_kernel(assignment, tree.right))

    """
    Convert parsed logic to a tree in NNF representation recursively.
    """
    def __convert_formula_to_tree(self):
        if not isinstance(self.formula, list):
            return self.formula
        else:
            if self.formula[0] == "not":
                if isinstance(self.formula[1], list) and self.formula[1][0] == "and":
                    self.left = Logic(["not", self.formula[1][1]])
                    self.right = Logic(["not", self.formula[1][2]])
                    return "or"
                elif isinstance(self.formula[1], list) and self.formula[1][0] == "or":
                    self.left = Logic(["not", self.formula[1][1]])
                    self.right = Logic(["not", self.formula[1][2]])
                    return "and"
                elif isinstance(self.formula[1], list) and self.formula[1][0] == "not":
                    equivalence = Logic(self.formula[1][1])
                    self.left = equivalence.left
                    self.right = equivalence.right
                    return equivalence.value
                else:
                    self.right = Logic(self.formula[1])
                    return self.formula[0]
            else:
                self.left = Logic(self.formula[1])
                self.right = Logic(self.formula[2])
                return self.formula[0]

Let's visualize one example: Here, the input formula is: $P = (\neg (x1 \wedge x2)\wedge x1)$. The list representation is: ["and", ["not", ["and", "x1", "x2"]], "x1"]. Using the Logic class, we convert the formula to a tree structure representing the equivalent formula in NNF: $P = (\neg x1 \vee \neg x2) \wedge x1$, as shown by the print_tree method, which prints the NNF representation in order.

In [2]:
logic = Logic(["and", ["not", ["and", "x1", "x2"]], "x1"])
logic.print_tree()

not
x1
or
not
x2
and
x1


## The DPLL Solver

The Davis-Putnam-Logemann-Loveland algorithm (DPLL) is a widely used algorithm in SAT solving. Its basic idea is recursive backtracking search of atom values that meet the satisfiability with some special heuristics.

Now, we briefly discuss the heuristics used in DPLL: There are two parts of the search algorithm in DPLL. In the recursive backtracking search, the algorithm first decides an assignment. Then, it deduces the assignment by substituting the atoms in the SAT formula with the assignment. If the valuation returns False, the algorithm backtracks to try another assignment if such new assignment is possible. If the algorithm runs out of assignments, the algorithm returns UNSAT. The heuristics in DPLL revolve around heuristics in deciding and deducing.

One heuristic in assigning is **Early termination**, which refers to terminating the algorithm if any realization of a variable leads to True or False of a SAT formula regardless the choice of any other variables. If the algorithmn returns False in the early termination, faster backtracking is encouraged. If the algorithm returns True in the early termination, on the other hand, faster path to a solution is found. To give a concrete example, $(A \vee B) \wedge (A \vee \neg C)$ is satisfied by {A = True}.

One heuristic for deciding assignments is **Pure literals**, which states that if all occurrences of a symbol in the clause (assuming a negation normal form) have the same sign accross the clause, we can guess that the symbol is True (if the symbol is positive) or False (if the symbol is negative). This can be done with the assumption that any "not" nodes far away from the leaf nodes are transformed to become the parents of the leaf nodes using the rules of De Morgan's Law and double negations (i.e. the SAT formula is in NNF). For instance, in $(A \vee B) \wedge (A \vee \neg C) \wedge (C \vee \neg B)$, the symbol A is pure and positive. By the heuristic, we should prioritize the decision of setting A to True over setting A to False in the search algorithm.

Another heuristic in assigning is **Unit Clauses**, which states that for any clause left with a single literal, we can simplify the clause by realizing that symbol. More broadly speaking, assignments to variables can lead to simplifications. For instance, in the case of {A = False} with $(A \vee B) \wedge (A \vee \neg C)$, the clause can be simplified to $(B) \wedge (\neg C)$
.

The above heuristics are what we implement in the DPLL solver. There are some more advanced heuristics that can increase the efficiency of the solver that we do not cover in our implementation.

Now, we begin with our function to simplify a Boolean formula following necessary heuristics.

In [3]:
def simplify(target, partial_assignment):
    """
    This function simplifies a formula given an assignment.

    target: the formula.
    partial_assignment: A possible assignment. An instance for the formula
    ["and", "x1", "x2"] is {"x1" : 1}.
    """
    if not isinstance(target, list):
        # This is the case of a single variable as the target formula.
        if target in partial_assignment:
            if partial_assignment[target]:
                return "True"
            else:
                return "False"
        else:
            return target
    else:
        if target[0] == "or":
            left_simplified = simplify(target[1], partial_assignment)
            right_simplified = simplify(target[2], partial_assignment)
            # Perform simplification.
            # Early termination:
            if left_simplified == "True" or right_simplified == "True":
                return "True"
            # Unit clause:
            elif left_simplified == "False":
                return right_simplified
            elif right_simplified == "False":
                return left_simplified
            else:
                return ["or", left_simplified, right_simplified]
        elif target[0] == "and":
            left_simplified = simplify(target[1], partial_assignment)
            right_simplified = simplify(target[2], partial_assignment)
            # Perform simplification.
            # Early termination:
            if left_simplified == "False" or right_simplified == "False":
                return "False"
            # Unit clause:
            if left_simplified == "True":
                return right_simplified
            elif right_simplified == "True":
                return left_simplified
            else:
                return ["and", left_simplified, right_simplified]
        else:
            right_simplified = simplify(target[1], partial_assignment)
            if right_simplified == "False":
                return "True"
            elif right_simplified == "True":
                return "False"
            else:
                return ["not", right_simplified]

Let's try some examples to see the heuristics we mentioned earlier.

In [4]:
# Early termination:
# (A or B) and (A or not C)
target = ["and", ["or", "x0", "x1"], ["or", "x0", ["not", "x2"]]]
print(simplify(target, {"x0" : 1}))

True


In [5]:
# Unit clause:
# (A or B) and (A or not C)
print(simplify(target, {"x0" : 0}))

['and', 'x1', ['not', 'x2']]


Now, let's look into the recursive backtracking part. We only focus on the solver that requires a single solution in our tutorial. For more comprehensive version, feel free to look into our python implementation.

In [6]:
# This is the entry point of our solver.
def solve_kernel_with_heuristic(target, variable_list, cur_assignment, pure_positives, pure_negatives, solutions):
    """
    This is the kernel for the solve function. It implements recursive backtracking.

    target: The SAT formula to be solved.
    variable_list: a list of SAT atoms.
    cur_assignment: We keep track of our current assignment in the recursive backtracking here.
    pure_positives: a list of SAT atoms that are pure positives.
    pure_negatives: a list of SAT atoms that are pure negatives.
    solutions: We keep track of our solution(s) in this data structure along the recursive backtracking process.
    """
    if len(cur_assignment.keys()) >= len(variable_list):
        # Base case: Reaches the end of assignment.
        return False
    else:
        index = len(cur_assignment.keys())
        variable = variable_list[index]
        # Perform the heuristic on pure literals.
        if variable in pure_positives:
            return solve_single(index, variable, target, variable_list, cur_assignment, pure_positives,
                                pure_negatives, solutions, [1, 0])
        else:
            return solve_single(index, variable, target, variable_list, cur_assignment, pure_positives,
                                pure_negatives, solutions, [0, 1])


# Now, we check out the solving procedure.
def solve_single(index, variable, target, variable_list, cur_assignment, pure_positives, pure_negatives,
                 solutions, ordering):
    """
    This function handles the case with solving for one solution only.

    cur_assignment: We keep track of our current assignment in the recursive backtracking here.
    """
    # Assign ordering[0] to the variable and try to simplify.
    new_assignment = cur_assignment.copy()
    new_assignment[variable] = ordering[0]
    simplified = simplify(target, new_assignment)
    # Case True following assigning ordering[0] to the variable: Return the correct answer.
    if simplified == "True":
        # Complete assignment.
        for i in range(index + 1, len(variable_list)):
            new_assignment[variable_list[i]] = 1
        solutions.append(new_assignment)
        return True
    # Case False following assigning ordering[0] to the variable: Try switch to ordering[1] and re-evaluate.
    elif simplified == "False":
        new_assignment[variable] = ordering[1]
        simplified = simplify(target, new_assignment)
        # If the solver attains true, return the correct answer.
        return further_search(index, simplified, variable_list, new_assignment, pure_positives, pure_negatives,
                              solutions)
    else:
        # The solver is inconclusive after assigning ordering[0] to the variable. Try to further simplify the formula.
        if solve_kernel_with_heuristic(simplified, variable_list, new_assignment, pure_positives,
                                       pure_negatives, solutions):
            return True
        else:
            # In this case, the solver resolves to false/inconclusive for assigning ordering[0] to the variable.
            new_assignment[variable] = ordering[1]
            simplified = simplify(target, new_assignment)
            return further_search(index, simplified, variable_list, new_assignment, pure_positives, pure_negatives,
                                  solutions)


# We require a further search step, which we present here:
def further_search(index, simplified, variable_list, new_assignment, pure_positives,
                   pure_negatives, solutions):
    """
    This function is used in solve_single for further searching and serves to help clean the codes.
    """
    if simplified == "True":
        # Complete assignment.
        for i in range(index + 1, len(variable_list)):
            new_assignment[variable_list[i]] = 1
        solutions.append(new_assignment)
        return True
    elif simplified == "False":
        # The assignment must be false.
        return False
    else:
        return solve_kernel_with_heuristic(simplified, variable_list, new_assignment,
                                           pure_positives, pure_negatives, solutions)

Now, let's write a function to initiate the kernel.

In [7]:
def solve(tree):
    """
    This is the main entry point for solving the formula using DPLL.

    The input is a SAT formula in its tree representation.
    """
    # Call the recursive backtracking kernel.
    variable_list = list(tree.leaves)
    solutions = []
    # Tree Heuristic automatically enabled in this branch.
    cur_assignment = dict()
    target = tree.formula
    solve_kernel_with_heuristic(target, variable_list, cur_assignment, tree.pure_positives, tree.pure_negatives, solutions)
    # Detect pure literals if assignment_heuristic_enabled: To be implemented later
    if len(solutions) == 0:
            return "UNSAT"
    else:
        return solutions[0]

Let's try some examples:

In [8]:
# Example 1: x1 and x2
formula = ["and", "x1", "x2"]
tree = Logic(formula)
solve(tree)

{'x2': 1, 'x1': 1}

In [9]:
# Example 2: x1 and (not (x2 or x1))
formula = ["and", "x1", ["not", ["or", "x2", "x1"]]]
tree = Logic(formula)
solve(tree)

'UNSAT'

In [10]:
# Example 3: x1
formula = "x1"
tree = Logic(formula)
solve(tree)

{'x1': 1}